## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

### Шаг 1. Импорт данных. 

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('/datasets/data.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Можно заметить, что nan  значения присутствуют в двух столбцах, это 'total_income' и 'total_income'.

Также сразу бросаются в глаза проблемы со стажем и разный регистр в образовании.


### Шаг 2. Предобработка данных

### Обработка пропусков

Проверим, имеется ли зависимость между этими пропусками

In [6]:
data.loc[data['total_income'].isnull()].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

Из значений в предыдущей ячейке следует, что имеется прямая зависимость между nan в столбце с доходами и nan в столбце с днями работы.  

In [7]:
data.groupby(['income_type'])['total_income'].mean()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

In [8]:
data.groupby(['income_type'])['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Отклонение между медианой и средним у людей одной занятости не сильно отличается, значит можем заменить пропуски в столбце 'total_income' на медианные значения по категории занятости, к которой относится пользователь с пропуском

In [9]:
import math
def fill_nan_with_median(row):
  if math.isnan(row['total_income']):
    return data[data['income_type'] == row['income_type']]['total_income'].median()
  else:
    return row['total_income']

In [10]:
data['total_income']=data.apply(fill_nan_with_median, axis=1)

In [11]:
data.loc[data['total_income'].isnull()].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Также, можно обратить внимание, что в столбце с возрастом присутствуют значения равные 0, что мало похоже на правду. Создадим датафрейм с медианными значениями по категориям занятости граждан и заполним ими нулевые значения в столбце с возрастом

In [12]:
temp=[]
for income_type in data['income_type'].value_counts().index:
    temp.append([income_type, data[data['income_type']==income_type]['dob_years'].median()])

In [13]:
employment_type=pd.DataFrame(columns=['income_type', 'median_value'], data=temp)
del temp

In [14]:
employment_type

,income_type,median_value
0,сотрудник,39.0
1,компаньон,39.0
2,пенсионер,60.0
3,госслужащий,40.0
4,предприниматель,42.5
5,безработный,38.0
6,студент,22.0
7,в декрете,39.0


In [15]:
def update_dob_years(row):
  if row['dob_years']==0:
    return employment_type[employment_type['income_type']==row['income_type']]['median_value'].mean()
  else:
    return row['dob_years']


In [16]:
data['dob_years']=data.apply(update_dob_years, axis=1)

In [17]:
data[data['dob_years']==0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [18]:
data=data.drop(columns=['days_employed'])

### Вывод


Поскольку мы выявили явную зависимость между значениями количества рабочих дней и поступающих сумм денежных средств, можно предположить, что у данных лиц отсутствуют официальные подтверждения занятости, а следовательно и получаемых средствах.

Подходящим способом заполнить пропуски является мединное значение для группы занятости пользователя с пропущенным значением.

### Замена типов данных

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 1.8+ MB


In [20]:
data['total_income']=data['total_income'].astype('int')

Также значение -1 в столбце с количеством детей кажется не особо правдоподнобным, возможно произошел сбой или анкеты были не правильно обрадотаты.

In [21]:
def update_children(row):
    if row['children']==-1:
        return int(data[(data['family_status_id']==row['family_status_id'])&(data['education']==row['education'])&(data['income_type']==row['income_type'])]['children'].mean())
    else:
        return row['children']

In [22]:
data['children']=data.apply(update_children, axis=1)

In [23]:
data['children'].value_counts()

 0     14195
 1      4818
 2      2055
 3       330
 20       76
 4        41
 5         9
-1         1
Name: children, dtype: int64

In [24]:
data=data[data['children']!=-1]

### Вывод

В качестве функции для изменения типа данных была выбрана функция 'astype' поскольку нам надо изменить тип данных на int.

В качестве целевого стобца был выбран столбец 'total_income', поскольку можно пренебречь долями денежной единицы при таких больших значениях.

### Обработка дубликатов

In [25]:
data['education'] = data['education'].str.lower()

In [26]:
data.duplicated(['education_id', 'family_status_id', 'income_type', 'purpose','total_income']).sum()

1443

In [28]:
data.duplicated().sum()

71

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 11 columns):
children            21524 non-null int64
dob_years           21524 non-null float64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null int64
purpose             21524 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 2.0+ MB


In [30]:
data

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...
21520,1,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


In [31]:
data[data.duplicated()]

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,41.0,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
3290,0,58.0,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
4182,1,34.0,высшее,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,60.0,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,58.0,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...
20702,0,64.0,среднее,1,женат / замужем,0,F,пенсионер,0,118514,дополнительное образование
21032,0,60.0,среднее,1,женат / замужем,0,F,пенсионер,0,118514,заняться образованием
21132,0,47.0,среднее,1,женат / замужем,0,F,сотрудник,0,142594,ремонт жилью
21281,1,30.0,высшее,0,женат / замужем,0,F,сотрудник,0,142594,покупка коммерческой недвижимости


### Вывод

Дубликаты были выбраны основываясь на значениях основных столбцов.

Дубликаты могли появиться, например, из-за того, что пользователь подал несколько однотипных заявок.

### Лемматизация

In [32]:
from pymystem3 import Mystem
m = Mystem()

In [33]:
words_pool=set()
for query in data['purpose'].unique():
    for word in query.split(' '):
        try:
            words_pool.add(m.lemmatize(word)[0])
        except:
            continue

### Вывод

Лемматизация выполнялась с помощью библиотеки pymystem3. 
Выделялись основные уникальные значения из столбца 'purpose'. И далее составлялся список уникальных слов, которые встретились.

### Категоризация данных

In [34]:
words_pool

{'автомобиль',
 'высокий',
 'для',
 'дополнительный',
 'жилой',
 'жилье',
 'заниматься',
 'коммерческий',
 'на',
 'недвижимость',
 'образование',
 'операция',
 'подержать',
 'покупка',
 'получение',
 'приобретение',
 'проведение',
 'профильный',
 'ремонт',
 'с',
 'свадьба',
 'свой',
 'сдача',
 'сделка',
 'семья',
 'со',
 'собственный',
 'строительство',
 'сыграть'}

In [35]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          625
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Если внимательно взглянуть на количество уникальных слов встречающихся в столбце с целью кредита, можно выделить несколько основных категорий займа:


*   Недвижимость
*   Автомобили
*   Образование
*   Свадьба





In [36]:
def group_purpose(purpose):
    lemmas=m.lemmatize(purpose)
    if 'жилье' in lemmas or 'недвижимость' in lemmas:
        return 'недвижимость'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    else:
        return 'образование'

In [37]:
data['purpose_group']=data['purpose'].apply(group_purpose)

In [38]:
data['purpose_group'].value_counts()

недвижимость    10839
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose_group, dtype: int64

### Вывод

Проанализировав уникальновстречающиеся слова в столбце с целями, можно выделить 4 основные категории нужд для займа.

После группировки можно оценить правильность её выполнения просуммировав значения выполнения функции value_counts() для столбца 'purpose' по соответствующим категориям.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Чтобы определить есть ли зависимость между этими показателями давайте найдем процент возвращенных долгов у людей с детьми и без.

In [39]:
total_person_with_children=data[(data['children']!=0)&(data['debt']==0)]['education_id'].count()+data[(data['children']!=0)&(data['debt']==1)]['education_id'].count()

In [40]:
data[(data['children']!=0)&(data['debt']==0)]['education_id'].count()/total_person_with_children

0.9076272342747987

In [41]:
total_person_without_children=data[(data['children']==0)&(data['debt']==0)]['education_id'].count()+data[(data['children']==0)&(data['debt']==1)]['education_id'].count()

In [42]:
data[(data['children']==0)&(data['debt']==0)]['education_id'].count()/total_person_without_children

0.925044029587883

Можно заметить, что процент людей вернувших долг не сильно отличается между людьми с детьми и без, следовательно, ощутимой зависимости не наблюдается. Дл дополнительного исследования можем оценить аналогичный параметр для разного количества детей.

Для этого построим сводную таблицу по столбцам 'children' и 'debt', в качестве значений можем взять любой из других столбцов, поскольку нам важно толко количество, которое посчитаем функцией count.

In [43]:
pivot_data=data.pivot_table(index='children', columns='debt', aggfunc='count', values='family_status_id')

In [44]:
pivot_data[1][5]=0

In [45]:
pivot_data

debt,0,1
children,,
0,13131.0,1064.0
1,4374.0,444.0
2,1861.0,194.0
3,303.0,27.0
4,37.0,4.0
5,9.0,0.0
20,68.0,8.0


In [46]:
pivot_data['sum']=pivot_data[0]+pivot_data[1]

In [47]:
pivot_data[1]=pivot_data[1]/pivot_data['sum']
pivot_data[0]=pivot_data[0]/pivot_data['sum']

In [48]:
pivot_data

debt,0,1,sum
children,,,
0,0.925044,0.074956,14195.0
1,0.907846,0.092154,4818.0
2,0.905596,0.094404,2055.0
3,0.918182,0.081818,330.0
4,0.902439,0.097561,41.0
5,1.000000,0.000000,9.0
20,0.894737,0.105263,76.0


### Вывод

Проведя анализ, удалось выяснить, что явной зависимости между количеством детей и возвратом кредита в срок нет

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [49]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Обратим внимание, что в данных присутствует выборос, очевидно, его следуют удалить из выборки

In [50]:
data=data[data['family_status']!=1]

Заполним сводную таблицу, аналогично с предыдущим пунктом, только вместо количества детей будем учитывать семейное положение.

In [51]:
pivot_family_data=data.pivot_table(index='family_status', columns='debt', values='education_id', aggfunc='count')

In [52]:
pivot_family_data

debt,0,1
family_status,,
Не женат / не замужем,2539,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3789,388
женат / замужем,11449,931


Сделаем аналогичные операции с вычислением количества вернувших долг от общего числа в данной категории.

In [53]:
pivot_family_data['sum']=pivot_family_data[0]+pivot_family_data[1]
pivot_family_data[1]=pivot_family_data[1]/pivot_family_data['sum']
pivot_family_data[0]=pivot_family_data[0]/pivot_family_data['sum']

In [54]:
pivot_family_data

debt,0,1,sum
family_status,,,
Не женат / не замужем,0.902595,0.097405,2813
в разводе,0.928870,0.071130,1195
вдовец / вдова,0.934307,0.065693,959
гражданский брак,0.907110,0.092890,4177
женат / замужем,0.924798,0.075202,12380


### Вывод

По результатам анализа можно обратить внимание, что процент возвращенных долгов у каждой из категории граждан достаточно высок, вне зависимости от семейного положеня. Следовательно прямой зависимости не наблюдается.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [55]:
data['total_income'].min()

20667

In [56]:
data['total_income'].max()

2265604

In [57]:
data['total_income'].median()

142594.0

In [58]:
data['total_income'].mean()

165224.5587716038

Судя по значениям основных метрик, сильного смещения в распределении уровня доходов нет, значит можем разделить нашу выборку на 5 категорий в соответсвии с попаданием в определнный квантиль.

In [59]:
data['total_income'].quantile([.2, .4, .6, .8])

0.2     98656.2
0.4    132140.6
0.6    161146.0
0.8    214276.2
Name: total_income, dtype: float64

In [60]:
def define_income_type(income):
  if income < 96178.2:
    return 1
  elif income < 129404.6:
    return 2
  elif income < 163672.8:
    return 3
  elif income < 219791.0:
    return 4
  else:
    return 5

In [61]:
data['total_income_type']=data['total_income'].apply(define_income_type)

In [62]:
data['total_income_type']. value_counts()

3    4897
4    4323
2    4271
5    4017
1    4016
Name: total_income_type, dtype: int64

In [63]:
income_pivot=data.pivot_table(index='total_income_type', columns='debt', values='education_id', aggfunc='count')

In [64]:
income_pivot

debt,0,1
total_income_type,,
1,3695,321
2,3914,357
3,4477,420
4,3965,358
5,3732,285


### Вывод

Даже без вычисления доли возвращенных кредитов можно заметить, что у каждй из категорий примерно одинаковое количество возвраенных долговых обязательств, следовательно прямой зависимости нет.

- Как разные цели кредита влияют на его возврат в срок?

In [65]:
data['purpose_group'].value_counts()

недвижимость    10839
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose_group, dtype: int64

In [66]:
pivot_data=data.pivot_table(index='purpose_group', columns='debt', values='education_id', aggfunc='count')

In [67]:
pivot_data['sum']=pivot_data[0]+pivot_data[1]
pivot_data[1]=pivot_data[1]/pivot_data['sum']
pivot_data[0]=pivot_data[0]/pivot_data['sum']

In [68]:
pivot_data

debt,0,1,sum
purpose_group,,,
автомобиль,0.906605,0.093395,4315
недвижимость,0.927853,0.072147,10839
образование,0.908006,0.091994,4022
свадьба,0.920784,0.079216,2348


### Вывод

Проведя аналогичные шаги для вычисления числа возвразенных долгов можно заметь, что здесь вновь отсутствуют какая-либо зависимость между возвратом долга и целью кредитования.

### Шаг 4. Общий вывод

По данной выборке можно сделать несколько выводов, например

Большинство людей, оформляюших кредит, имеют среднее образование.
При этом наиболее частая причина для оформления кредита это операции с недвижимостью. Практически одинаковая доля долгов приходится на приобретение автомобилей и образование, и десятая часть от всех, оформляет кредит на свадьбу.
Доходы среди выборки распределены равномерно, несмотря на большой разброс между мксимальным значеним и медианным(и средним).

Какой-то явной зависимости между целями кредита/количеством детей/семейным полоэением/денежным обеспечением и выполнению обязательств по кредиту нет.